# Make input files from failed sequence

In [36]:
failed = pd.read_csv('failed_all.csv')
failed.head()

,Accession,Sequence,Class
0,BbCD00385184,ATGACCCTGAAAACCATGACGCTGGCTTTTGCCGTAACCGGCCTTG...,0.0
1,CaCD00344362,ATGCTTTTTATTATAAAATACTATGAACCTAAGAACCTATATGAGC...,0.0
2,DhCD00338941,ATGCCCCAGGAGGGGCTGAGGTTTGTAGTGCAGCACCATCTGGCCC...,0.0
3,EcCD00331370,ATGCCGGGCAACAGCCCGCATTATGGGCGTTGGCCTCAACACGATT...,0.0
4,LbCD00598846,ATGTTAGGACGTTCTGATGTCGGCCGAACCGACACCTTGATGGTTG...,0.0


In [93]:
failed['Sequence'][0]

'ATGACCCTGAAAACCATGACGCTGGCTTTTGCCGTAACCGGCCTTGGCGTGTTGGCGGGATGCTCATCGCCCTCGGTGGTACAGCAGCGCGATGGCAGTCAGGTCGTGACGCCCGACGAGCCGAAGTACAACGAGGACACCGGCTTCTACGAGTACGAGAAGGACGGCCACAAGGTCCAGATGAACAAGGACGACGTCAAGACGATCGAGGAGGTCAAGCTCGAGCACCACCACCACCACCACTGA'

In [37]:
def translate(seq):
    length = (len(seq)- len(seq)%3)
    split_func = lambda seq, n: [seq[i:i+n] for\
                                    i in range(0, length, n)]
    codons = split_func(seq, 3)
    aa = ''
    for c in codons:
        aa+=data.CODON_TO_AA[c]
    return aa

In [39]:
failed['AA'] = failed['Sequence'].apply(translate)
failed.head()

,Accession,Sequence,Class,AA
0,BbCD00385184,ATGACCCTGAAAACCATGACGCTGGCTTTTGCCGTAACCGGCCTTG...,0.0,MTLKTMTLAFAVTGLGVLAGCSSPSVVQQRDGSQVVTPDEPKYNED...
1,CaCD00344362,ATGCTTTTTATTATAAAATACTATGAACCTAAGAACCTATATGAGC...,0.0,MLFIIKYYEPKNLYEHILFILKHLCRSKSMILNPQKLSVDFGNGIT...
2,DhCD00338941,ATGCCCCAGGAGGGGCTGAGGTTTGTAGTGCAGCACCATCTGGCCC...,0.0,MPQEGLRFVVQHHLARRDHYDLRLEWDGAMLSWAVPKGPSYNTRDK...
3,EcCD00331370,ATGCCGGGCAACAGCCCGCATTATGGGCGTTGGCCTCAACACGATT...,0.0,MPGNSPHYGRWPQHDFTSLKKLRPQSVTSRIQPGSDVIVCAEMDEQ...
4,LbCD00598846,ATGTTAGGACGTTCTGATGTCGGCCGAACCGACACCTTGATGGTTG...,0.0,MLGRSDVGRTDTLMVATINPTKETAYLTSIPRDTRVTVGSGVNASV...


In [56]:
for index, val in enumerate(failed['AA']):
    encryptID = ['aaSeq', 'flagHash', 'gcTarget', 'exclSeqList', 'repeatNum',
       'allrepeatNum', 'c2aa', 'aa2c', 'cPair2aaPair', 'cCountRef',
       'aaCountRef', 'cPairCountRef', 'aaPairCountRef', 'GArunParam',
       'randSeed']
    XXXXXXXX = [val,'IC:1;CC:1;HiddenStop:1;CAI:1;ExclusionSequence:1;GC:1;Repeat:1;allRepeat:1;',\
               41,'YGAUCC,AAGCUU,GUCGAC,GAAUUC','5:2;','8:2;',c2aa,aa2c,cPair2aaPair,cCountRef,\
               aaCountRef,cPairCountRef,aaPairCountRef,'0:0:0;',12345678]
    in_df = pd.DataFrame({'encryptID':encryptID, 'XXXXXXXX':XXXXXXXX})
    fname = 'inputs/'+str(index)+'_param'
    in_df.to_csv(fname, index=None, sep='\t')

In [67]:
#change dir to your location


for i in range(len(failed)):
    
    run(['/home/bikash/Documents/pub/COOl/inputs/co_ubuntu.exe',str(i)],
    stdout=PIPE, cwd = '/home/bikash/Documents/pub/COOl/inputs/')

In [81]:
def cool(i):
    run(['/home/bikash/Documents/pub/COOl/inputs/co_ubuntu.exe',str(i)],
    stdout=PIPE, cwd = '/home/bikash/Documents/pub/COOl/inputs/')
    
def read_results(i):
    #extract result with maximum codon cost fitness
    fname = '/home/bikash/Documents/pub/COOl/inputs/'+str(i)+'_results'
    res = pd.read_csv(fname, skiprows=5, sep='\t')
    res.sort_values(['CC'], ascending=False, inplace=True)
    res.reset_index(inplace=True)
    return res["Sequence"][0]
    

In [85]:

from multiprocessing import Pool, cpu_count
pools = Pool(cpu_count()-1)
pool_results = []
for result in pools.imap_unordered(cool, \
                                     list(range(len(failed)))):
    pool_results.append(result)


pools.close()
pools.join()

#pools = Pool(cpu_count()-1)
#pool_results1 = []
#for result in pools.imap_unordered(read_results, \
#                                     list(range(len(failed)))):
#    pool_results1.append(result)


#pools.close()
#pools.join()



In [98]:
optimised_seq = []
for i in list(range(len(failed))):
              optimised_seq.append(read_results(i))

In [99]:
cool_optimised = pd.DataFrame({'Sequence':optimised_seq})
cool_optimised.head()

,Sequence
0,AUGACGUUAAAAACCAUGACCCUGGCAUUCGCUGUGACUGGUCUGG...
1,AUGCUGUUCAUUAUUAAAUACUACGAACCUAAAAACCUGUAUGAAC...
2,AUGCCGCAGGAAGGCUUACGCUUCGUUGUUCAACAUCAUCUUGCCC...
3,AUGCCGGGUAAUUCUCCGCACUACGGUAGAUGGCCCCAACAUGAUU...
4,AUGCUGGGCCGCAGCGACGUCGGUCGCACGGAUACGUUAAUGGUGG...


In [100]:
cool_optimised.to_csv('COOL_optimised_TT.csv', index=None)